#  Bloque 1 

In [1]:
# ==========================================
# BLOQUE 1: SETUP ENTORNO KAGGLE
# ==========================================
# @title 1. Configuración Inicial (Kaggle Environment)
import os
import sys
import subprocess
import torch
import gc
import warnings

# Suprimir advertencias no críticas de PyTorch/Matplotlib
warnings.filterwarnings("ignore")

def setup_kaggle_env():
    print("🏗️ Configurando entorno Kaggle...")
    
    # 1. Instalar SAM 2 y Geoespacial (Si no existen)
    try:
        import sam2
    except ImportError:
        print("⏳ Instalando SAM 2 y librerías geoespaciales...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "git+https://github.com/facebookresearch/segment-anything-2.git"])
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "shapely", "geopandas", "rasterio", "opencv-python-headless"])
    
    print("✅ Entorno listo.")

setup_kaggle_env()

import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import ipywidgets as widgets
from IPython.display import display, clear_output, FileLink
import io
import requests
from shapely.geometry import Polygon
import geopandas as gpd

# Configuración de Hardware (Soporte P100/T4)
if torch.cuda.is_available():
    DEVICE = "cuda"
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✅ GPU Activa: {gpu_name}")
    
    # Optimización: bfloat16 solo si es Ampere (A100) o Turing (T4). 
    # Si te toca una Pascal (P100), usamos float16 o float32 estándar.
    if "T4" in gpu_name or "A100" in gpu_name:
        torch.autocast(device_type="cuda", dtype=torch.bfloat16).__enter__()
        if torch.cuda.get_device_properties(0).major >= 8:
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True
else:
    DEVICE = "cpu"
    print("⚠️ ADVERTENCIA: CPU detectada. Se recomienda activar GPU en Settings > Accelerator.")

# Directorios de trabajo Kaggle
WORK_DIR = "/kaggle/working"
OUTPUT_DIR = os.path.join(WORK_DIR, "output_tiles")
os.makedirs(OUTPUT_DIR, exist_ok=True)


🏗️ Configurando entorno Kaggle...
⏳ Instalando SAM 2 y librerías geoespaciales...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires pylibraft-cu12==25.6.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires rmm-cu12==25.6.*, but you have rmm-cu12 25.2.0 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 70.1 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
ydata-profiling 4.17.0 requires numpy<2.2,>=1.16.0, but you have numpy 2.2.6 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2

✅ Entorno listo.
✅ GPU Activa: Tesla T4


#  Bloque 2

In [2]:

# ==========================================
# BLOQUE 2: CARGA DE IMAGEN
# ==========================================
# @title 2. Carga de Mapa Geológico
original_image_global = None
processed_image_global = None

def image_loader_ui():
    style = {'description_width': 'initial'}
    
    # UI Elements
    lbl_title = widgets.HTML("<h3>📍 Carga de Imagen</h3>")
    source_w = widgets.Dropdown(options=['GitHub URL', 'Subir Archivo'], value='GitHub URL', description='Fuente:', style=style)
    url_w = widgets.Text(value="https://raw.githubusercontent.com/DalemberV/Geodigit/main/Digitalizacion_Automatica/Mapas%20geol%C3%B3gicos/TuMapa.jpg", placeholder='URL...', layout=widgets.Layout(width='60%'))
    upload_w = widgets.FileUpload(accept='image/*', multiple=False)
    btn_load = widgets.Button(description="Cargar", button_style='primary')
    out = widgets.Output()

    def on_load(b):
        global original_image_global
        with out:
            clear_output()
            try:
                if source_w.value == 'GitHub URL':
                    print(f"Descargando: {url_w.value} ...")
                    resp = requests.get(url_w.value, stream=True).raw
                    original_image_global = Image.open(resp).convert("RGB")
                else:
                    if not upload_w.value: return print("❌ Selecciona un archivo.")
                    f_info = next(iter(upload_w.value.values())) # Kaggle file upload struct
                    original_image_global = Image.open(io.BytesIO(f_info['content'])).convert("RGB")
                
                print(f"✅ Imagen cargada: {original_image_global.size}")
                # Mostrar thumbnail
                thumb = original_image_global.copy()
                thumb.thumbnail((400, 400))
                display(thumb)
                
            except Exception as e:
                print(f"❌ Error: {e}")

    btn_load.on_click(on_load)
    display(widgets.VBox([lbl_title, widgets.HBox([source_w, url_w, upload_w, btn_load]), out]))

image_loader_ui()


# Bloque 3

In [3]:
# ==========================================
# BLOQUE 3: PREPROCESAMIENTO
# ==========================================
# @title 3. Ajuste Visual (Preprocesamiento)
def preprocessing_ui():
    style = {'description_width': 'initial'}
    lbl_title = widgets.HTML("<h3>🎨 Ajuste de Imagen (Brillo/Contraste)</h3>")
    
    b_slider = widgets.FloatSlider(value=1.0, min=0.5, max=2.0, step=0.1, description='Brillo')
    c_slider = widgets.FloatSlider(value=1.0, min=0.5, max=2.0, step=0.1, description='Contraste')
    s_slider = widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, description='Nitidez')
    btn_apply = widgets.Button(description="Confirmar y Guardar", button_style='success')
    out_prev = widgets.Output()

    def update_view(change=None):
        if original_image_global is None: return
        with out_prev:
            clear_output(wait=True)
            img = original_image_global.copy()
            img = ImageEnhance.Brightness(img).enhance(b_slider.value)
            img = ImageEnhance.Contrast(img).enhance(c_slider.value)
            img = ImageEnhance.Sharpness(img).enhance(s_slider.value)
            
            # Preview rápido
            w, h = img.size
            ratio = 500 / w
            plt.figure(figsize=(8, 4))
            plt.imshow(img.resize((500, int(h*ratio))))
            plt.axis('off')
            plt.title("Preview")
            plt.show()

    def apply_filters(b):
        global processed_image_global
        if original_image_global is None: return
        img = original_image_global.copy()
        img = ImageEnhance.Brightness(img).enhance(b_slider.value)
        img = ImageEnhance.Contrast(img).enhance(c_slider.value)
        processed_image_global = ImageEnhance.Sharpness(img).enhance(s_slider.value)
        print("✅ Imagen preprocesada lista para segmentar.")

    b_slider.observe(update_view, names='value')
    c_slider.observe(update_view, names='value')
    s_slider.observe(update_view, names='value')
    btn_apply.on_click(apply_filters)

    display(widgets.VBox([lbl_title, widgets.HBox([b_slider, c_slider, s_slider]), out_prev, btn_apply]))

preprocessing_ui()

# Bloque 4

In [4]:
# ==========================================
# BLOQUE 4: CARGA MODELO SAM 2
# ==========================================
# @title 4. Inicialización del Modelo
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

if not os.path.exists("sam2_hiera_large.pt"):
    print("⬇️ Bajando pesos SAM 2 Large...")
    os.system("wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt")

print("⚙️ Cargando Modelo a GPU...")
torch.set_grad_enabled(False)
sam2_model = build_sam2("sam2_hiera_l.yaml", "sam2_hiera_large.pt", device=DEVICE, apply_postprocessing=False)
print("✅ Modelo cargado en memoria (Weights loaded).")

⬇️ Bajando pesos SAM 2 Large...
⚙️ Cargando Modelo a GPU...
✅ Modelo cargado en memoria (Weights loaded).


# Bloque 5

In [5]:
# ==========================================
# BLOQUE 5: CONSOLA DE SEGMENTACIÓN POR TILE
# ==========================================
# @title 5. Explorador Dinámico de Tiles (Kaggle Edition)

class TileExplorer:
    def __init__(self, image, model):
        self.image = np.array(image)
        self.model = model
        self.H, self.W = self.image.shape[:2]
        self.tile_size = 1024
        self.tiles = self._generate_tiles()
        self.current_idx = 0
        self.current_masks = []
        self.current_polys_gdf = None
        
        # UI Widgets
        self.out_display = widgets.Output()
        self.out_status = widgets.Output()
        
        # Controles de Navegación
        self.btn_prev = widgets.Button(description="< Anterior", icon='arrow-left')
        self.btn_next = widgets.Button(description="Siguiente >", icon='arrow-right')
        self.lbl_idx = widgets.Label(value=f"Tile: 1 / {len(self.tiles)}")
        
        # Controles de Parámetros SAM (Dinámicos)
        self.sl_points = widgets.IntSlider(value=64, min=16, max=128, step=16, description='Puntos/Lado')
        self.sl_iou = widgets.FloatSlider(value=0.7, min=0.1, max=1.0, step=0.05, description='Calidad IOU')
        self.sl_stability = widgets.FloatSlider(value=0.9, min=0.5, max=1.0, step=0.01, description='Estabilidad')
        
        # Acciones
        self.btn_run = widgets.Button(description="⚡ EJECUTAR SEGMENTACIÓN", button_style='warning', layout=widgets.Layout(width='98%'))
        self.btn_save = widgets.Button(description="💾 Guardar Tile", button_style='success')
        
        # Visualización
        self.chk_dark = widgets.Checkbox(value=False, description='Fondo Oscuro')
        self.chk_bw_filter = widgets.Checkbox(value=True, description='Ignorar Blanco/Negro')

        # Eventos
        self.btn_prev.on_click(self.prev_tile)
        self.btn_next.on_click(self.next_tile)
        self.btn_run.on_click(self.run_segmentation)
        self.btn_save.on_click(self.save_tile)
        
        self.render_ui()
        self.show_current_tile_raw() # Mostrar inicial sin procesar

    def _generate_tiles(self):
        # Generar coordenadas sin overlap para almacenamiento (o con, según prefieras para visualización)
        # Aquí usamos overlap para visualización pero guardamos el ID
        coords = []
        stride = int(self.tile_size * 0.9) # 10% overlap
        for y in range(0, self.H, stride):
            for x in range(0, self.W, stride):
                x_end = min(x + self.tile_size, self.W)
                y_end = min(y + self.tile_size, self.H)
                x_start = max(0, x_end - self.tile_size)
                y_start = max(0, y_end - self.tile_size)
                coords.append((x_start, y_start, x_end, y_end))
        return coords

    def get_current_tile_img(self):
        x1, y1, x2, y2 = self.tiles[self.current_idx]
        return self.image[y1:y2, x1:x2]

    def prev_tile(self, b):
        if self.current_idx > 0:
            self.current_idx -= 1
            self.update_state()

    def next_tile(self, b):
        if self.current_idx < len(self.tiles) - 1:
            self.current_idx += 1
            self.update_state()
            
    def update_state(self):
        self.lbl_idx.value = f"Tile: {self.current_idx + 1} / {len(self.tiles)}"
        self.current_masks = [] # Limpiar máscaras anteriores
        self.current_polys_gdf = None
        self.show_current_tile_raw()

    def run_segmentation(self, b):
        with self.out_status: clear_output(); print("⏳ Procesando...")
        
        # 1. Configurar Generador con parámetros ACTUALES de los sliders
        mask_generator = SAM2AutomaticMaskGenerator(
            model=self.model,
            points_per_side=self.sl_points.value,
            points_per_batch=64, # Fijo para P100/T4
            pred_iou_thresh=self.sl_iou.value,
            stability_score_thresh=self.sl_stability.value,
            stability_score_offset=0.7,
            crop_n_layers=0,
            min_mask_region_area=500
        )
        
        # 2. Inferencia
        tile_img = self.get_current_tile_img()
        masks = mask_generator.generate(tile_img)
        
        # 3. Filtrado B/N (Si está activo)
        self.current_masks = []
        for m in masks:
            if self.chk_bw_filter.value:
                seg = m['segmentation'].astype(np.uint8)
                mean_val = cv2.mean(tile_img, mask=seg)[:3]
                if all(c > 220 for c in mean_val) or all(c < 30 for c in mean_val):
                    continue
            self.current_masks.append(m)
            
        with self.out_status: clear_output(); print(f"✅ Detectados: {len(self.current_masks)} polígonos.")
        self.plot_results()
        
        # Limpieza VRAM
        torch.cuda.empty_cache()

    def plot_results(self):
        with self.out_display:
            clear_output(wait=True)
            tile_img = self.get_current_tile_img()
            
            fig, ax = plt.subplots(figsize=(10, 10))
            
            if self.chk_dark.value:
                ax.imshow(np.full_like(tile_img, 30)) # Gris oscuro
            else:
                ax.imshow(tile_img)
            
            # Dibujar máscaras
            if len(self.current_masks) > 0:
                sorted_anns = sorted(self.current_masks, key=(lambda x: x['area']), reverse=True)
                for ann in sorted_anns:
                    m = ann['segmentation']
                    img = np.ones((m.shape[0], m.shape[1], 4))
                    color = np.concatenate([np.random.random(3), [0.6]])
                    img[:,:,0] = color[0]
                    img[:,:,1] = color[1]
                    img[:,:,2] = color[2]
                    img[:,:,3] = 0.5 
                    ax.imshow(np.dstack((img[:,:,:3], m * img[:,:,3])))
                    
                    # Contorno
                    contours, _ = cv2.findContours(m.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    for cnt in contours:
                        if len(cnt) > 2:
                            cnt = cnt.squeeze()
                            if cnt.ndim == 2:
                                ax.plot(cnt[:, 0], cnt[:, 1], linewidth=1, color='white', alpha=0.8)
            
            ax.set_title(f"Tile {self.current_idx} | Params: Pts={self.sl_points.value}, IOU={self.sl_iou.value}")
            ax.axis('off')
            plt.show()

    def show_current_tile_raw(self):
        with self.out_display:
            clear_output(wait=True)
            plt.figure(figsize=(6, 6))
            plt.imshow(self.get_current_tile_img())
            plt.title(f"Vista Original Tile {self.current_idx}")
            plt.axis('off')
            plt.show()

    def save_tile(self, b):
        if not self.current_masks:
            with self.out_status: print("❌ No hay máscaras para guardar. Ejecuta primero.")
            return
            
        # Vectorizar
        x1, y1, _, _ = self.tiles[self.current_idx]
        polys = []
        for ann in self.current_masks:
            seg = ann['segmentation'].astype(np.uint8)
            contours, _ = cv2.findContours(seg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for cnt in contours:
                if len(cnt) > 2:
                    # Guardamos coordenadas GLOBALES
                    cnt_global = cnt + [x1, y1]
                    p = Polygon(cnt_global.reshape(-1, 2))
                    if p.is_valid and p.area > 50:
                        polys.append(p)
        
        if polys:
            gdf = gpd.GeoDataFrame(geometry=polys)
            # Nombre de archivo único
            fname = f"tile_{self.current_idx}_pos_{x1}_{y1}.geojson"
            path = os.path.join(OUTPUT_DIR, fname)
            gdf.to_file(path, driver='GeoJSON')
            with self.out_status: print(f"💾 Guardado: {fname}")
        else:
            with self.out_status: print("⚠️ No se generaron polígonos válidos.")

    def render_ui(self):
        # Layout
        ctrl_params = widgets.VBox([
            widgets.Label("🔧 Parámetros SAM 2 (Ajustables por Tile):"),
            self.sl_points, self.sl_iou, self.sl_stability,
            self.chk_bw_filter, self.chk_dark,
            self.btn_run, self.btn_save
        ])
        
        nav = widgets.HBox([self.btn_prev, self.lbl_idx, self.btn_next])
        
        full_ui = widgets.HBox([
            widgets.VBox([nav, self.out_display]), # Izquierda: Visualización
            widgets.VBox([ctrl_params, self.out_status]) # Derecha: Controles
        ])
        
        display(full_ui)

# Iniciar la App
if processed_image_global is not None:
    explorer = TileExplorer(processed_image_global, sam2_model)
else:
    print("⚠️ Primero carga y preprocesa la imagen en los bloques anteriores.")

⚠️ Primero carga y preprocesa la imagen en los bloques anteriores.
